# 03 - 订单与订阅相关表探索

目标：探索订单和订阅相关的5张表结构、数据量和关键字段分布。

**目标表**: `` `order` ``, `subscribe`, `set_meal`, `order_amount_info`, `cloud_info`

In [1]:
import sys
sys.path.insert(0, '..')

from src.db.connector import DBConnector
from config.db_config import BRAND_DB_MAP
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_rows', 50)

/Users/petercheng/Library/Python/3.9/lib/python/site-packages/paramiko/pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  "cipher": algorithms.TripleDES,
/Users/petercheng/Library/Python/3.9/lib/python/site-packages/paramiko/transport.py:253: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  "class": algorithms.TripleDES,


In [2]:
# 要探索的表
ORDER_TABLES = ['`order`', 'subscribe', 'set_meal', 'order_amount_info', 'cloud_info']

## 1. 建立连接

In [3]:
db = DBConnector(brand='osaio')
db.connect()
print(f'已连接: {db.database}')

已连接: bi_center


## 2. OSAIO (bi_center) - 表探索

In [4]:
def explore_table(db, table_name):
    """探索单张表的结构、数据量、样本和关键字段分布"""
    display_name = table_name.strip('`')
    print(f"\n{'='*70}")
    print(f"表: {display_name}")
    print('='*70)
    
    try:
        # 字段结构
        desc = db.query_df(f'DESCRIBE {table_name}')
        print(f"\n--- 字段结构 ({len(desc)} 列) ---")
        display(desc)
        
        # 数据量
        count = db.query_df(f'SELECT COUNT(*) as cnt FROM {table_name}')
        print(f"\n--- 数据量: {count['cnt'].iloc[0]:,} 行 ---")
        
        # 样本数据
        sample = db.query_df(f'SELECT * FROM {table_name} LIMIT 10')
        print(f"\n--- 样本数据 (前10行) ---")
        display(sample)
        
        # 关键字段值分布
        print(f"\n--- 关键字段值分布 ---")
        for col in desc['Field']:
            # 对 status, type, description 等分类字段做值分布
            if any(kw in col.lower() for kw in ['status', 'type', 'description', 'currency', 'platform', 'unit']):
                try:
                    dist = db.query_df(f'SELECT `{col}`, COUNT(*) as cnt FROM {table_name} GROUP BY `{col}` ORDER BY cnt DESC LIMIT 20')
                    print(f"\n  [{col}] 值分布 (top 20):")
                    display(dist)
                except Exception as e:
                    print(f"  [{col}] 查询失败: {e}")
        
        return count['cnt'].iloc[0]
    except Exception as e:
        print(f"查询失败: {e}")
        return 0

In [5]:
# OSAIO: 探索所有订单相关表
osaio_counts = {}
for table in ORDER_TABLES:
    cnt = explore_table(db, table)
    osaio_counts[table.strip('`')] = cnt


表: order



--- 字段结构 (26 列) ---


,Field,Type,Null,Key,Default,Extra
0,uid,varchar(20),NO,MUL,None,
1,id,int unsigned,NO,PRI,None,auto_increment
2,uuid,varchar(64),NO,MUL,,
3,appid,varchar(16),NO,MUL,,
4,order_id,varbinary(60),NO,MUL,None,
5,account,varchar(320),NO,,None,
6,subscribe_id,varchar(20),NO,MUL,,
7,product_id,varchar(50),NO,MUL,,
8,status,tinyint unsigned,NO,,0,
9,description,varchar(1024),NO,,,



--- 数据量: 672,678 行 ---



--- 样本数据 (前10行) ---


...


--- 关键字段值分布 ---



  [status] 值分布 (top 20):


,status,cnt
0,0,355770
1,1,247479
2,2,2165



  [status_msg] 值分布 (top 20):


,status_msg,cnt
0,,604248
1,orderVerification check PayDenied,1166


### 2.1 OSAIO - order 表关键字段深入探索

In [6]:
# order 表 status 字段
print('=== order.status 分布 ===')
display(db.query_df('SELECT status, COUNT(*) as cnt FROM `order` GROUP BY status ORDER BY cnt DESC'))

# order 表 description 字段 (试用/付费识别)
print('\n=== order.description 分布 (top 30) ===')
display(db.query_df('SELECT description, COUNT(*) as cnt FROM `order` GROUP BY description ORDER BY cnt DESC LIMIT 30'))

# order 表 product_id 分布
print('\n=== order.product_id 分布 (top 20) ===')
display(db.query_df('SELECT product_id, COUNT(*) as cnt FROM `order` GROUP BY product_id ORDER BY cnt DESC LIMIT 20'))

=== order.status 分布 ===


,status,cnt
0,1,597725
1,0,72756
2,6,1455
3,5,692
4,4,50



=== order.description 分布 (top 30) ===


,description,cnt
0,,557667
1,FREE_TRIAL,49929
2,Trial: 14 DAY,46727
3,Promotion: 14 DAY,10404
4,merge payment order,4371
5,FreeCloud: 99 Year,1024
6,后台:46 指定开通,531
7,后台:97 指定开通,299
8,upgrade,198
9,升级重复扣费,175



=== order.product_id 分布 (top 20) ===


,product_id,cnt
0,2d45e3725a5340568d134b65d0c7caa2,218643
1,eb34f5845642ed10ab2d91703baf28e3,95033
2,18e75ca570f308d530eb2060928af51e,52040
3,,49837
4,52b72d358d4811eb93c0067a3e6cf430,38091
5,25ef29e69266a18e4e62a9abfb24906e,34851
6,71d7bf50084b13308df8baa5d0ab67b4,32127
7,c9f452e0f08e80c1093a82616cc946a4,16868
8,d90d40f341f05593daf9d4c4702f8bee,16332
9,689860a9db2aac4e0375d2edbaf73faf,16306


# subscribe 表关键字段分布
print('=== subscribe.status 分布 ===')
display(db.query_df('SELECT status, COUNT(*) as cnt FROM subscribe GROUP BY status ORDER BY cnt DESC'))

print('\n=== subscribe.cycles_unit 分布 ===')
display(db.query_df('SELECT cycles_unit, COUNT(*) as cnt FROM subscribe GROUP BY cycles_unit ORDER BY cnt DESC'))

print('\n=== subscribe.support_dev_num 分布 ===')
display(db.query_df('SELECT support_dev_num, COUNT(*) as cnt FROM subscribe GROUP BY support_dev_num ORDER BY cnt DESC'))

print('\n=== subscribe.currency 分布 ===')
display(db.query_df('SELECT currency, COUNT(*) as cnt FROM subscribe GROUP BY currency ORDER BY cnt DESC'))


In [7]:
# order_amount_info 关键字段
print('=== order_amount_info 金额统计 ===')
display(db.query_df("""
    SELECT 
           COUNT(*) as cnt,
           ROUND(AVG(amount_cny), 2) as avg_amount_cny,
           ROUND(MIN(amount_cny), 2) as min_amount_cny,
           ROUND(MAX(amount_cny), 2) as max_amount_cny,
           ROUND(AVG(transaction_fee_cny), 2) as avg_fee_cny
    FROM order_amount_info
    WHERE amount_cny > 0
"""))


=== order_amount_info 金额统计 ===


,cnt,avg_amount_cny,min_amount_cny,max_amount_cny,avg_fee_cny
0,364909,46.95,0.1,1133.55,3.91


In [8]:
# cloud_info 关键字段
print('=== cloud_info.status 分布 ===')
display(db.query_df('SELECT status, COUNT(*) as cnt FROM cloud_info GROUP BY status ORDER BY cnt DESC'))

print('\n=== cloud_info 样本 (含时间字段) ===')
display(db.query_df('SELECT * FROM cloud_info LIMIT 10'))

=== cloud_info.status 分布 ===


,status,cnt
0,0,355770
1,1,247479
2,2,2165



=== cloud_info 样本 (含时间字段) ===


,id,uid,uuid,schema,order_id,start_time,end_time,file_time,status,status_msg,is_event,is_delete,level,service,created_at,updated_at
0,1,497eef498a2f3a59,ea28d8b7ea3299ec0fdf408e30585e9b,,193685ea509411eb94590a3be2346914,1610017057,1611446400,2,0,,1,0,0,,2023-10-27 01:34:59,2024-03-21 04:12:28
1,2,497eef498a2f3a59,e7025c5e4decf920a976ad53da6a6aa6,,e745748e817e11eb89060a3be2346914,1615395561,1616803200,2,0,,1,0,0,,2023-10-27 01:34:59,2024-03-21 04:12:28
2,3,31cff23c67ab5310,f3f34b6a9589e54b2270f6871f6929e1,,admin_2021092406554920300268261,1632495349,1635120000,30,0,,0,0,0,,2023-10-27 01:34:59,2024-03-21 04:12:28
3,4,580f234fee4bd4bd,e8ff29e385b47568a886e92e5ae05d37,,admin_202109301017057200271177,1633025825,1664582400,30,0,,0,0,0,,2023-10-27 01:34:59,2024-03-21 04:12:28
4,5,c4e26934b3fcda3e,6ce2f1da5998a2aa77de55cb589af26c,,0cd7fcc1400911ec9047027a48cd9df2,1636320415,1637712000,2,0,,1,0,0,,2023-10-27 01:34:59,2024-03-21 04:12:28
5,6,68128ab0197d49a8,94f4a61b2e365af8c4ab139b6e185c45,,9cbadccf426e11ec9047027a48cd9df2,1636583938,1637971200,2,0,,1,0,0,,2023-10-27 01:34:59,2024-03-21 04:12:28
6,7,b33103b8600ec07a,06276758bf60fcd68401359a7716d901,,df80f26642ec11ec9047027a48cd9df2,1636638167,1638057600,2,0,,1,0,0,,2023-10-27 01:34:59,2024-03-21 04:12:28
7,8,7547032f792f32d3,16124e076b924f503df8d0b4e7d4109e,,8e92bd49448711eca7f20a044883c176,1636814878,1639526400,15,0,,1,0,0,,2023-10-27 01:34:59,2024-03-21 04:12:28
8,9,74d76396501f7b33,264ba6469b4d6187c113c24b50489403,,f397cadb46cf11ec9047027a48cd9df2,1637065550,1638489600,2,0,,1,0,0,,2023-10-27 01:34:59,2024-03-21 04:12:28
9,10,74d76396501f7b33,264ba6469b4d6187c113c24b50489403,,8f319cec46db11eca7f20a044883c176,1638489600,1641081600,15,0,,1,0,0,,2023-10-27 01:34:59,2024-04-11 09:26:50


### 2.2 OSAIO - 表关联关系验证

In [9]:
# 验证表关联关系 (使用子查询避免全表JOIN)
print('=== order → subscribe 关联验证 ===')
total = db.query_df("SELECT COUNT(*) as cnt FROM `order`")['cnt'].iloc[0]
matched = db.query_df("SELECT COUNT(*) as cnt FROM `order` o WHERE EXISTS (SELECT 1 FROM subscribe s WHERE s.subscribe_id = o.subscribe_id)")['cnt'].iloc[0]
print(f"  total_orders: {total:,}, matched_subscribe: {matched:,}, no_subscribe: {total-matched:,}")

print('\n=== order → set_meal 关联验证 ===')
matched = db.query_df("SELECT COUNT(*) as cnt FROM `order` o WHERE EXISTS (SELECT 1 FROM set_meal sm WHERE sm.code = o.product_id)")['cnt'].iloc[0]
print(f"  total_orders: {total:,}, matched_meal: {matched:,}, no_meal: {total-matched:,}")

print('\n=== order → order_amount_info 关联验证 ===')
matched = db.query_df("SELECT COUNT(*) as cnt FROM `order` o WHERE EXISTS (SELECT 1 FROM order_amount_info oai WHERE oai.order_int_id = o.id)")['cnt'].iloc[0]
print(f"  total_orders: {total:,}, has_amount: {matched:,}, no_amount: {total-matched:,}")

print('\n=== order → cloud_info 关联验证 ===')
matched = db.query_df("SELECT COUNT(*) as cnt FROM `order` o WHERE EXISTS (SELECT 1 FROM cloud_info ci WHERE ci.order_id = o.order_id)")['cnt'].iloc[0]
print(f"  total_orders: {total:,}, has_cloud: {matched:,}, no_cloud: {total-matched:,}")

=== order → subscribe 关联验证 ===


  total_orders: 672,678, matched_subscribe: 583,419, no_subscribe: 89,259

=== order → set_meal 关联验证 ===


  total_orders: 672,678, matched_meal: 622,841, no_meal: 49,837

=== order → order_amount_info 关联验证 ===


  total_orders: 672,678, has_amount: 389,785, no_amount: 282,893

=== order → cloud_info 关联验证 ===


  total_orders: 672,678, has_cloud: 597,420, no_cloud: 75,258


## 3. Nooie (nooie_bi_center) - 表探索

In [10]:
db.switch_database('nooie')
print(f'已切换到: {db.database}')

已切换到: nooie_bi_center


In [11]:
# Nooie: 探索所有订单相关表
nooie_counts = {}
for table in ORDER_TABLES:
    cnt = explore_table(db, table)
    nooie_counts[table.strip('`')] = cnt


表: order



--- 字段结构 (25 列) ---


,Field,Type,Null,Key,Default,Extra
0,uid,varchar(20),NO,MUL,None,
1,id,int unsigned,NO,PRI,None,auto_increment
2,uuid,varchar(64),NO,,,
3,appid,varchar(16),NO,MUL,,
4,order_id,char(32),NO,MUL,None,
5,account,varchar(320),NO,,None,
6,subscribe_id,varchar(20),NO,MUL,,
7,product_id,varchar(50),NO,MUL,,
8,status,tinyint unsigned,NO,,0,
9,description,varchar(1024),NO,,,



--- 数据量: 252,398 行 ---

--- 样本数据 (前10行) ---


,uid,id,uuid,appid,order_id,account,subscribe_id,product_id,status,description,pay_type,is_sub,is_test,product_name,amount,transaction_fee,currency,submit_time,pay_time,paypal_token,redirect_url,notify_url,payer_id,created_at,updated_at
0,1f6273eadb7f81fb,1,98644875b96ef6f17c70f812c6de7260,4adcd2139621b1ef,2019071603474569700000001,,0,f908d48a09bc83f8cbbaeaec0877683e,0,,4,1,0,3天存两天,0.15,0.0,EUR,1563248865,1563248865,0,,,,2024-06-29 04:40:05,2024-06-29 04:40:05
1,1f6273eadb7f81fb,2,98644875b96ef6f17c70f812c6de7260,4adcd2139621b1ef,2019071603475022800000001,,0,f908d48a09bc83f8cbbaeaec0877683e,0,,4,1,0,3天存两天,0.15,0.0,EUR,1563248870,1563248870,0,,,,2024-06-29 04:40:05,2024-06-29 04:40:05
2,1f6273eadb7f81fb,3,98644875b96ef6f17c70f812c6de7260,4adcd2139621b1ef,2019071603475691500000001,,0,f908d48a09bc83f8cbbaeaec0877683e,0,,4,1,0,3天存两天,0.15,0.0,EUR,1563248876,1563248876,0,,,,2024-06-29 04:40:05,2024-06-29 04:40:05
3,1f6273eadb7f81fb,4,98644875b96ef6f17c70f812c6de7260,4adcd2139621b1ef,2019071603523585700000001,,I-BT6CN8HJH3VC,f908d48a09bc83f8cbbaeaec0877683e,5,,4,1,1,3天存两天,0.15,0.0,EUR,1563249155,1563249155,0,,,,2024-06-29 04:40:05,2024-06-29 04:40:05
4,f6f5af4a2fdc291a,5,e902dfb5f3a724f5d8d307f1af802141,4adcd2139621b1ef,2019072217474493700000005,,I-46RXUGE1YM4D,31dbe5f12b27b58d50b2304cc00fd2d6,1,,4,1,0,7-Day PLAYBACK,0.10,0.0,EUR,1563817664,1563817664,0,,,,2024-06-29 04:40:05,2024-06-29 04:40:05
5,df1a35bae5eb974f,6,0cbc08f62fd9ce81804aed48e93f64dd,4adcd2139621b1ef,2019072217525956700000002,,I-LKP73UJJEBL3,31dbe5f12b27b58d50b2304cc00fd2d6,1,,4,1,0,7-Day PLAYBACK,0.10,0.0,EUR,1563817979,1563817979,0,,,,2024-06-29 04:40:05,2024-06-29 04:40:05
6,be3b40f74a6600dc,7,ea896b04eb26657529bf4114c74e1a77,4adcd2139621b1ef,admin_2019072606262939100000012,,0,61a50f5d3a3b961feca5158d838b2029,1,后台: nooie@apemans.com 指定开通,0,0,0,one month cloud experience,0.00,0.0,EUR,1564122389,1564122389,0,,,,2024-06-29 04:40:05,2024-06-29 04:40:05
7,ac46fa0f291c97e6,8,b4b4f00a7960a9127ca858db54ce3eb9,4adcd2139621b1ef,2019072612562491800000022,,0,31dbe5f12b27b58d50b2304cc00fd2d6,0,,4,1,0,7-Day PLAYBACK,0.10,0.0,EUR,1564145784,1564145784,0,,,,2024-06-29 04:40:05,2024-06-29 04:40:05
8,ac46fa0f291c97e6,9,b4b4f00a7960a9127ca858db54ce3eb9,4adcd2139621b1ef,2019072613412835900000022,,I-9497AYANSKGK,31dbe5f12b27b58d50b2304cc00fd2d6,1,,4,1,0,7-Day PLAYBACK,0.10,0.0,EUR,1564148488,1564148488,0,,,,2024-06-29 04:40:05,2024-06-29 04:40:05
9,f6f5af4a2fdc291a,10,308899e8717f82d0c86da209a3046228,4adcd2139621b1ef,2019073019145210900000005,,I-AF0NMSFM14BE,31dbe5f12b27b58d50b2304cc00fd2d6,1,,4,1,0,7-Day PLAYBACK,0.10,0.0,EUR,1564514092,1564514092,0,,,,2024-06-29 04:40:05,2024-06-29 04:40:05


...


--- 关键字段值分布 ---



  [status] 值分布 (top 20):


,status,cnt
0,0,137690
1,1,88463



  [status_msg] 值分布 (top 20):


,status_msg,cnt
0,,225806
1,orderVerification check PayDenied,347


In [12]:
# Nooie: order 表深入探索
print('=== Nooie order.status 分布 ===')
display(db.query_df('SELECT status, COUNT(*) as cnt FROM `order` GROUP BY status ORDER BY cnt DESC'))

print('\n=== Nooie order.description 分布 (top 30) ===')
display(db.query_df('SELECT description, COUNT(*) as cnt FROM `order` GROUP BY description ORDER BY cnt DESC LIMIT 30'))

print('\n=== Nooie order.product_id 分布 (top 20) ===')
display(db.query_df('SELECT product_id, COUNT(*) as cnt FROM `order` GROUP BY product_id ORDER BY cnt DESC LIMIT 20'))

=== Nooie order.status 分布 ===


,status,cnt
0,1,222760
1,0,27376
2,6,1747
3,5,436
4,2,78
5,4,1



=== Nooie order.description 分布 (top 30) ===


,description,cnt
0,,150892
1,14 days Video History Event Recording Monthly,29100
2,3 days Video History CVR Recording Monthly,24036
3,30 days Video History Event Recording Monthly,7960
4,3 days Video History CVR Recording Annual,7515
5,7 days video history CVR recording monthly,6201
6,Trial: 14 DAY,5542
7,14 days Video History Event Recording Annual,5411
8,30 days video history event recording monthly pro,3913
9,30 days Video History Event Recording Annual,1784



=== Nooie order.product_id 分布 (top 20) ===


,product_id,cnt
0,93cdb3d2daf9311960e6c3eef2b7df59,73680
1,1aa8342b9e834a3230f909ab74f3b289,24059
2,7fbeed7fd22f11eaa280067a3e6cf430,19647
3,e0b1a2094ff811eb99f5067a3e6cf430,19322
4,3f30783cd22f11eaa280067a3e6cf430,16169
5,4d543ceca28112ac6528d33f999e0658,12988
6,347dec49e3fe0d98c5ea7a7804c78535,11281
7,dbeece497081cfdbc997c215bd3737c9,8989
8,bf227e3fa29aca05979bfc36e28f5144,8167
9,a5df0684d21b11eaa280067a3e6cf430,7567


In [13]:
# Nooie: order_amount_info 金额统计
print('=== Nooie order_amount_info 金额统计 ===')
display(db.query_df("""
    SELECT 
           COUNT(*) as cnt,
           ROUND(AVG(amount_cny), 2) as avg_amount_cny,
           ROUND(MIN(amount_cny), 2) as min_amount_cny,
           ROUND(MAX(amount_cny), 2) as max_amount_cny,
           ROUND(AVG(transaction_fee_cny), 2) as avg_fee_cny
    FROM order_amount_info
    WHERE amount_cny > 0
"""))


=== Nooie order_amount_info 金额统计 ===


,cnt,avg_amount_cny,min_amount_cny,max_amount_cny,avg_fee_cny
0,102766,54.15,6.5,1118.35,5.4


### 3.1 Nooie - 表关联关系验证

In [14]:
# Nooie 关联验证 (使用子查询避免全表JOIN)
total = db.query_df("SELECT COUNT(*) as cnt FROM `order`")['cnt'].iloc[0]

print('=== Nooie order → subscribe 关联 ===')
matched = db.query_df("SELECT COUNT(*) as cnt FROM `order` o WHERE EXISTS (SELECT 1 FROM subscribe s WHERE s.subscribe_id = o.subscribe_id)")['cnt'].iloc[0]
print(f"  total_orders: {total:,}, matched_subscribe: {matched:,}, no_subscribe: {total-matched:,}")

print('\n=== Nooie order → set_meal 关联 ===')
matched = db.query_df("SELECT COUNT(*) as cnt FROM `order` o WHERE EXISTS (SELECT 1 FROM set_meal sm WHERE sm.code = o.product_id)")['cnt'].iloc[0]
print(f"  total_orders: {total:,}, matched_meal: {matched:,}, no_meal: {total-matched:,}")

print('\n=== Nooie order → order_amount_info 关联 ===')
matched = db.query_df("SELECT COUNT(*) as cnt FROM `order` o WHERE EXISTS (SELECT 1 FROM order_amount_info oai WHERE oai.order_int_id = o.id)")['cnt'].iloc[0]
print(f"  total_orders: {total:,}, has_amount: {matched:,}, no_amount: {total-matched:,}")

print('\n=== Nooie order → cloud_info 关联 ===')
matched = db.query_df("SELECT COUNT(*) as cnt FROM `order` o WHERE EXISTS (SELECT 1 FROM cloud_info ci WHERE ci.order_id = o.order_id)")['cnt'].iloc[0]
print(f"  total_orders: {total:,}, has_cloud: {matched:,}, no_cloud: {total-matched:,}")

=== Nooie order → subscribe 关联 ===


  total_orders: 252,398, matched_subscribe: 205,656, no_subscribe: 46,742

=== Nooie order → set_meal 关联 ===


  total_orders: 252,398, matched_meal: 252,398, no_meal: 0

=== Nooie order → order_amount_info 关联 ===


  total_orders: 252,398, has_amount: 108,450, no_amount: 143,948

=== Nooie order → cloud_info 关联 ===


  total_orders: 252,398, has_cloud: 221,595, no_cloud: 30,803


## 4. 数据量汇总对比

In [15]:
summary = []
for table in ORDER_TABLES:
    display_name = table.strip('`')
    summary.append({
        '表': display_name,
        'OSAIO': f"{osaio_counts.get(display_name, 'N/A'):,}" if isinstance(osaio_counts.get(display_name), int) else 'N/A',
        'Nooie': f"{nooie_counts.get(display_name, 'N/A'):,}" if isinstance(nooie_counts.get(display_name), int) else 'N/A',
    })

summary_df = pd.DataFrame(summary)
print('=== 订单相关表数据量汇总 ===')
display(summary_df)

=== 订单相关表数据量汇总 ===


,表,OSAIO,Nooie
0,order,N/A,N/A
1,subscribe,N/A,N/A
2,set_meal,N/A,N/A
3,order_amount_info,N/A,N/A
4,cloud_info,N/A,N/A


In [16]:
db.close()
print('连接已关闭')

连接已关闭
